### **Curating body_back1a.h5ad**

Article:  Multi-scale spatial mapping of cell populations across anatomical sites in healthy human skin and basal cell carcinoma

DOI: https://doi.org/10.1101/2023.08.08.551504

Data Source : https://spatial-skin-atlas.cellgeni.sanger.ac.uk

##### **Mount farm**

mount-farm

##### **Packages required for curation**

In [1]:
#Import all packages required for curation

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import scipy
from tqdm import tqdm
from scipy import sparse
from scipy.sparse import csr_matrix
import anndata as ad
import os
import subprocess
import math

### **Curation Schema**

##### **X (Matrix Layers)**

##### **AnnData object**

In [3]:
# Load the AnnData object

In [4]:
adata = sc.read_h5ad('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Data/WSSKNKCLsp10446621.h5ad')

In [5]:
# View the AnnData object

In [6]:
adata

AnnData object with n_obs × n_vars = 2418 × 15888
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes', 'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc', 'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes', 'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts', 'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC', 'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes', 'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1'
    uns: 'leiden', 'log1p', 'neighbors', 'pca', 'spatial', 'umap'
    obsm: 'X_spatial'

##### **X- expression matrix**

In [7]:
# View the expression matrix of the anndata object

In [8]:
adata.X

array([[-0.04701956, -0.03877327, -0.10538654, ..., -0.03759922,
        -0.14403716, -0.09172469],
       [-0.04701956, -0.03877327, -0.10538654, ..., -0.03759922,
        -0.14403716, -0.09172469],
       [-0.04701956, -0.03877327, -0.10538654, ..., -0.03759922,
        -0.14403716, -0.09172469],
       ...,
       [-0.04701956, -0.03877327, -0.10538654, ..., -0.03759922,
        -0.14403716, -0.09172469],
       [-0.04701956, -0.03877327, -0.10538654, ..., -0.03759922,
        -0.14403716, -0.09172469],
       [-0.04701956, -0.03877327,  5.5956106 , ..., -0.03759922,
        -0.14403716, -0.09172469]], dtype=float32)

In [9]:
#data type of adata.X

In [10]:
type(adata.X)

numpy.ndarray

In [11]:
#convert to csr_matrix

In [12]:
adata.X = csr_matrix(adata.X)

In [13]:
adata.X

<2418x15888 sparse matrix of type '<class 'numpy.float32'>'
	with 38417184 stored elements in Compressed Sparse Row format>

In [14]:
# Print the matrix to check whether they are normalized counts or raw counts. if the matrix has floating numbers,they are normalized counts.if they are integers, they are raw counts.

In [15]:
print(adata.X)

  (0, 0)	-0.047019556
  (0, 1)	-0.038773272
  (0, 2)	-0.10538654
  (0, 3)	-0.09742447
  (0, 4)	-0.05425526
  (0, 5)	-0.051723678
  (0, 6)	-0.22798301
  (0, 7)	-0.1154247
  (0, 8)	-0.17113191
  (0, 9)	-0.105408296
  (0, 10)	-0.3415758
  (0, 11)	-0.1304191
  (0, 12)	-0.13663623
  (0, 13)	-0.14975104
  (0, 14)	-0.12348838
  (0, 15)	-0.19210231
  (0, 16)	-0.0985333
  (0, 17)	-0.30237147
  (0, 18)	-0.11590331
  (0, 19)	-0.12048806
  (0, 20)	-0.21133025
  (0, 21)	-0.1468588
  (0, 22)	-0.35655484
  (0, 23)	-0.10862948
  (0, 24)	-0.30358008
  :	:
  (2417, 15863)	-0.11190085
  (2417, 15864)	-0.051263932
  (2417, 15865)	-0.06882972
  (2417, 15866)	-0.10263304
  (2417, 15867)	-0.05811565
  (2417, 15868)	-0.16681562
  (2417, 15869)	-0.03901318
  (2417, 15870)	-0.13898727
  (2417, 15871)	1.0616865
  (2417, 15872)	0.9923684
  (2417, 15873)	1.3738687
  (2417, 15874)	1.2862372
  (2417, 15875)	0.95176
  (2417, 15876)	1.1192449
  (2417, 15877)	1.0982032
  (2417, 15878)	0.92610466
  (2417, 15879)	1.93028

##### **Raw counts matrix**

In [16]:
# If X has normalized counts, check for the raw counts matrix.

In [17]:
#Here the raw counts are provided in a separate object, load the raw counts matrix

In [18]:
araw= sc.read_10x_h5('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Raw/all/WSSKNKCLsp10446621/filtered_feature_bc_matrix.h5')

/nfs/users/nfs_s/sa32/anaconda/envs/shibla_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [19]:
# view raw object

In [20]:
araw

AnnData object with n_obs × n_vars = 2418 × 36601
    var: 'gene_ids', 'feature_types', 'genome'

In [21]:
# view raw matrix

In [22]:
araw.X

<2418x36601 sparse matrix of type '<class 'numpy.float32'>'
	with 1528364 stored elements in Compressed Sparse Row format>

In [23]:
print(araw.X)

  (0, 219)	1.0
  (0, 846)	1.0
  (0, 930)	1.0
  (0, 1237)	1.0
  (0, 1752)	1.0
  (0, 1853)	1.0
  (0, 1957)	1.0
  (0, 2066)	1.0
  (0, 2098)	1.0
  (0, 2123)	1.0
  (0, 2141)	1.0
  (0, 2146)	1.0
  (0, 2221)	1.0
  (0, 2461)	1.0
  (0, 2735)	1.0
  (0, 3870)	1.0
  (0, 3919)	1.0
  (0, 4261)	1.0
  (0, 5145)	1.0
  (0, 5310)	1.0
  (0, 5475)	1.0
  (0, 5759)	1.0
  (0, 5765)	1.0
  (0, 6077)	1.0
  (0, 6163)	1.0
  :	:
  (2417, 36101)	1.0
  (2417, 36112)	1.0
  (2417, 36172)	1.0
  (2417, 36237)	1.0
  (2417, 36243)	1.0
  (2417, 36298)	1.0
  (2417, 36352)	1.0
  (2417, 36371)	1.0
  (2417, 36373)	1.0
  (2417, 36378)	3.0
  (2417, 36398)	2.0
  (2417, 36401)	13.0
  (2417, 36406)	2.0
  (2417, 36559)	34.0
  (2417, 36560)	28.0
  (2417, 36561)	52.0
  (2417, 36562)	49.0
  (2417, 36563)	1.0
  (2417, 36564)	36.0
  (2417, 36565)	81.0
  (2417, 36566)	33.0
  (2417, 36567)	3.0
  (2417, 36568)	67.0
  (2417, 36569)	8.0
  (2417, 36571)	38.0


In [24]:
# since the raw object is combined one, extract the raw counts for this dataset 

##### **Variables(var)**

In [25]:
#View the var of anndata and raw object

In [26]:
adata.var

""
AL627309.1
AL627309.5
LINC01409
LINC01128
FAM41C
...
MT-CYB
AC145212.1
AL592183.1
AC007325.4


In [27]:
araw.var

,gene_ids,feature_types,genome
MIR1302-2HG,ENSG00000243485,Gene Expression,GRCh38
FAM138A,ENSG00000237613,Gene Expression,GRCh38
OR4F5,ENSG00000186092,Gene Expression,GRCh38
AL627309.1,ENSG00000238009,Gene Expression,GRCh38
AL627309.3,ENSG00000239945,Gene Expression,GRCh38
...,...,...,...
AC141272.1,ENSG00000277836,Gene Expression,GRCh38
AC023491.2,ENSG00000278633,Gene Expression,GRCh38
AC007325.1,ENSG00000276017,Gene Expression,GRCh38
AC007325.4,ENSG00000278817,Gene Expression,GRCh38


In [28]:
#Ensembl IDs

In [29]:
gene_info = pd.read_table('/lustre/scratch127/cellgen/cellgeni/shibla/ref_files/2020A.gene_names.tsv')

In [30]:
gene_info

,ensembl_ids,gene,version
0,ENSG00000243485,MIR1302-2HG,2020A
1,ENSG00000237613,FAM138A,2020A
2,ENSG00000186092,OR4F5,2020A
3,ENSG00000238009,AL627309.1,2020A
4,ENSG00000239945,AL627309.3,2020A
...,...,...,...
36596,ENSG00000277836,AC141272.1,2020A
36597,ENSG00000278633,AC023491.2,2020A
36598,ENSG00000276017,AC007325.1,2020A
36599,ENSG00000278817,AC007325.4,2020A


In [31]:
#create a dictionary with gene symbols and ensembl ids from the gene information file

In [32]:
gene_info_genesym_to_ensembl = dict(zip(gene_info['gene'],gene_info['ensembl_ids']))

In [33]:
gene_info_genesym_to_ensembl

{'MIR1302-2HG': 'ENSG00000243485',
 'FAM138A': 'ENSG00000237613',
 'OR4F5': 'ENSG00000186092',
 'AL627309.1': 'ENSG00000238009',
 'AL627309.3': 'ENSG00000239945',
 'AL627309.2': 'ENSG00000239906',
 'AL627309.5': 'ENSG00000241860',
 'AL627309.4': 'ENSG00000241599',
 'AP006222.2': 'ENSG00000286448',
 'AL732372.1': 'ENSG00000236601',
 'OR4F29': 'ENSG00000284733',
 'AC114498.1': 'ENSG00000235146',
 'OR4F16': 'ENSG00000284662',
 'AL669831.2': 'ENSG00000229905',
 'LINC01409': 'ENSG00000237491',
 'FAM87B': 'ENSG00000177757',
 'LINC01128': 'ENSG00000228794',
 'LINC00115': 'ENSG00000225880',
 'FAM41C': 'ENSG00000230368',
 'AL645608.6': 'ENSG00000272438',
 'AL645608.2': 'ENSG00000230699',
 'AL645608.4': 'ENSG00000241180',
 'LINC02593': 'ENSG00000223764',
 'SAMD11': 'ENSG00000187634',
 'NOC2L': 'ENSG00000188976',
 'KLHL17': 'ENSG00000187961',
 'PLEKHN1': 'ENSG00000187583',
 'PERM1': 'ENSG00000187642',
 'AL645608.7': 'ENSG00000272512',
 'HES4': 'ENSG00000188290',
 'ISG15': 'ENSG00000187608',
 'AL6

In [34]:
#Store ensembl ids in a new column in adata.var by matching gene symbols and ensembl ids from the gene information file

In [35]:
adata.var['gene_symbols'] = adata.var_names

In [36]:
araw.var['gene_symbols'] = araw.var_names

In [37]:
araw.var_names = araw.var['gene_ids']

In [38]:
adata.var['ensembl_id'] = adata.var['gene_symbols'].map(gene_info_genesym_to_ensembl)

In [39]:
adata.var

,gene_symbols,ensembl_id
AL627309.1,AL627309.1,ENSG00000238009
AL627309.5,AL627309.5,ENSG00000241860
LINC01409,LINC01409,ENSG00000237491
LINC01128,LINC01128,ENSG00000228794
FAM41C,FAM41C,ENSG00000230368
...,...,...
MT-CYB,MT-CYB,ENSG00000198727
AC145212.1,AC145212.1,ENSG00000277400
AL592183.1,AL592183.1,ENSG00000273748
AC007325.4,AC007325.4,ENSG00000278817


In [40]:
adata.var['ensembl_id'].isna().sum()

14

In [41]:
adata.var_names = adata.var['ensembl_id']

In [42]:
adata.var

,gene_symbols,ensembl_id
ensembl_id,,
ENSG00000238009,AL627309.1,ENSG00000238009
ENSG00000241860,AL627309.5,ENSG00000241860
ENSG00000237491,LINC01409,ENSG00000237491
ENSG00000228794,LINC01128,ENSG00000228794
ENSG00000230368,FAM41C,ENSG00000230368
...,...,...
ENSG00000198727,MT-CYB,ENSG00000198727
ENSG00000277400,AC145212.1,ENSG00000277400
ENSG00000273748,AL592183.1,ENSG00000273748


In [43]:
del adata.var['gene_symbols']
del adata.var['ensembl_id']

In [44]:
del araw.var['gene_ids']
del araw.var['feature_types']
del araw.var['genome']
del araw.var['gene_symbols']

In [45]:
adata.var.index= adata.var.index.drop_duplicates()

In [46]:
adata.var

""
ensembl_id
ENSG00000238009
ENSG00000241860
ENSG00000237491
ENSG00000228794
ENSG00000230368
...
ENSG00000198727
ENSG00000277400
ENSG00000273748


In [47]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


In [48]:
# load the approved genes file

In [49]:
approved_genes = pd.read_csv('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/Endometrium_reference_integrated_atlas/genes_approved.csv')

In [50]:
# Create a dictionary from the approved genes file using the symbols and feature id columns.

In [51]:
genedict = {key: 1 for key in list(approved_genes.feature_id)}

In [52]:
genedict

{'ERCC-00002': 1,
 'ERCC-00003': 1,
 'ERCC-00004': 1,
 'ERCC-00009': 1,
 'ERCC-00012': 1,
 'ERCC-00013': 1,
 'ERCC-00014': 1,
 'ERCC-00016': 1,
 'ERCC-00017': 1,
 'ERCC-00019': 1,
 'ERCC-00022': 1,
 'ERCC-00024': 1,
 'ERCC-00025': 1,
 'ERCC-00028': 1,
 'ERCC-00031': 1,
 'ERCC-00033': 1,
 'ERCC-00034': 1,
 'ERCC-00035': 1,
 'ERCC-00039': 1,
 'ERCC-00040': 1,
 'ERCC-00041': 1,
 'ERCC-00042': 1,
 'ERCC-00043': 1,
 'ERCC-00044': 1,
 'ERCC-00046': 1,
 'ERCC-00048': 1,
 'ERCC-00051': 1,
 'ERCC-00053': 1,
 'ERCC-00054': 1,
 'ERCC-00057': 1,
 'ERCC-00058': 1,
 'ERCC-00059': 1,
 'ERCC-00060': 1,
 'ERCC-00061': 1,
 'ERCC-00062': 1,
 'ERCC-00067': 1,
 'ERCC-00069': 1,
 'ERCC-00071': 1,
 'ERCC-00073': 1,
 'ERCC-00074': 1,
 'ERCC-00075': 1,
 'ERCC-00076': 1,
 'ERCC-00077': 1,
 'ERCC-00078': 1,
 'ERCC-00079': 1,
 'ERCC-00081': 1,
 'ERCC-00083': 1,
 'ERCC-00084': 1,
 'ERCC-00085': 1,
 'ERCC-00086': 1,
 'ERCC-00092': 1,
 'ERCC-00095': 1,
 'ERCC-00096': 1,
 'ERCC-00097': 1,
 'ERCC-00098': 1,
 'ERCC-000

In [53]:
len(genedict)

119799

In [54]:
# Filter out the genes which are not in the approved genes file

In [55]:
var_to_keep_adata = [x for x in adata.var_names if (x in genedict)]
var_to_keep_araw = [x for x in araw.var_names if (x in genedict)]

In [56]:
len(var_to_keep_adata)

15824

In [57]:
len(var_to_keep_araw)

36390

In [58]:
# Modify the anndata object by filtering out the filtered genes. copy the index column values to a new column called gene_symbols

In [59]:
adata = adata[:, var_to_keep_adata].copy()
araw = araw[:, var_to_keep_araw].copy()

In [60]:
#  View the var

In [61]:
adata.var

""
ensembl_id
ENSG00000238009
ENSG00000241860
ENSG00000237491
ENSG00000228794
ENSG00000230368
...
ENSG00000198727
ENSG00000277400
ENSG00000273748


In [62]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


feature is filtered

In [63]:
def add_zero():
	global adata
	global araw
	if araw.shape[1] > adata.shape[1]:
		genes_add = [x for x in araw.var.index.to_list() if x not in adata.var.index.to_list()]
		new_matrix = sparse.csr_matrix((adata.X.data, adata.X.indices, adata.X.indptr), shape = araw.shape)
		all_genes = adata.var.index.to_list()
		all_genes.extend(genes_add)
		new_var = pd.DataFrame(index=all_genes)
		new_var = pd.merge(new_var, araw.var, left_index=True, right_index=True, how='left')
		new_var['feature_is_filtered'] = False
		new_var.loc[genes_add, 'feature_is_filtered'] = True
		new_adata = ad.AnnData(X=new_matrix, obs=adata.obs, var=new_var, uns=adata.uns, obsm=adata.obsm)
		if adata.layers:
			for layer in adata.layers:
				new_layer = sparse.csr_matrix((adata.layers[layer].data, adata.layers[layer].indices, adata.layers[layer].indptr), shape = araw.shape)
				new_adata.layers[layer] = new_layer
		new_adata = new_adata[:,araw.var.index.to_list()]
		new_adata.var = new_adata.var.merge(adata.var, left_index=True, right_index=True, how='left')
		adata = new_adata
	else:
		adata.var['feature_is_filtered'] = False

In [64]:
add_zero()

In [65]:
# view var

In [66]:
adata.var

,feature_is_filtered
ENSG00000243485,True
ENSG00000237613,True
ENSG00000186092,True
ENSG00000238009,False
ENSG00000239945,True
...,...
ENSG00000277836,True
ENSG00000278633,True
ENSG00000276017,True
ENSG00000278817,False


In [67]:
list(adata.var['feature_is_filtered'].unique())

[True, False]

In [68]:
True_count = (adata.var['feature_is_filtered']== True).sum()

In [69]:
True_count

20566

In [70]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


#### **Observations(obs) (Cell metadata)**

In [71]:
#view obs

In [72]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_PTGDS+ fibroblasts,c2l_MigDC,c2l_Neuronal_SchwannC,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,173,195,8,4.102564,15,7.692308,AAACACCAATAACTGC-1,...,0.009635,0.105004,0.018953,0.024605,0.006612,3,0.171487,1.714548,0.061199,0.005679
AAACAGCTTTCAGAAG-1,1,43,9,216,261,19,7.279693,50,19.157087,AAACAGCTTTCAGAAG-1,...,0.003326,0.003190,0.009110,0.002446,0.347454,1,4.056033,5.330975,0.054002,0.001896
AAACAGGGTCTATATT-1,1,47,13,494,668,38,5.688622,90,13.473053,AAACAGGGTCTATATT-1,...,0.006169,0.001033,0.028893,0.065812,1.291950,4,6.572667,2.795006,0.270538,0.001760
AAACATGGTGAGAGGA-1,1,62,0,435,523,22,4.206501,51,9.751434,AAACATGGTGAGAGGA-1,...,0.014965,0.001391,0.032490,0.081424,0.027660,5,0.172046,1.047911,0.270653,0.004278
AAACCGGGTAGGTACC-1,1,42,28,347,453,47,10.375277,53,11.699780,AAACCGGGTAGGTACC-1,...,0.005588,0.000547,0.017277,0.032265,0.819287,4,3.862053,3.408283,0.243512,0.000508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,185,210,6,2.857143,18,8.571429,TTGTTCAGTGTGCTAC-1,...,0.002684,0.000940,0.010030,0.140575,0.026521,1,0.603950,0.187112,0.014475,0.001555
TTGTTGTGTGTCAAGA-1,1,31,77,143,175,4,2.285714,18,10.285714,TTGTTGTGTGTCAAGA-1,...,0.023014,0.002091,0.008887,0.032770,0.001563,3,0.266120,0.018888,0.014429,0.010205
TTGTTTCACATCCAGG-1,1,58,42,75,79,6,7.594936,5,6.329114,TTGTTTCACATCCAGG-1,...,0.001632,0.000330,0.002047,0.003116,0.010596,0,0.520835,0.022958,0.005394,0.000805


In [73]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1'],
      dtype='object')

#### **assay_ontology_term_id**

In [74]:
# add the assay_ontology_term_id column

In [75]:
adata.obs['assay_ontology_term_id'] = ['EFO:0010961'] * len(adata.obs)

In [76]:
# change datatype of the column

In [77]:
adata.obs['assay_ontology_term_id'] = adata.obs['assay_ontology_term_id'].astype('category')

In [78]:
# view adata.obs

In [79]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_MigDC,c2l_Neuronal_SchwannC,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,173,195,8,4.102564,15,7.692308,AAACACCAATAACTGC-1,...,0.105004,0.018953,0.024605,0.006612,3,0.171487,1.714548,0.061199,0.005679,EFO:0010961
AAACAGCTTTCAGAAG-1,1,43,9,216,261,19,7.279693,50,19.157087,AAACAGCTTTCAGAAG-1,...,0.003190,0.009110,0.002446,0.347454,1,4.056033,5.330975,0.054002,0.001896,EFO:0010961
AAACAGGGTCTATATT-1,1,47,13,494,668,38,5.688622,90,13.473053,AAACAGGGTCTATATT-1,...,0.001033,0.028893,0.065812,1.291950,4,6.572667,2.795006,0.270538,0.001760,EFO:0010961
AAACATGGTGAGAGGA-1,1,62,0,435,523,22,4.206501,51,9.751434,AAACATGGTGAGAGGA-1,...,0.001391,0.032490,0.081424,0.027660,5,0.172046,1.047911,0.270653,0.004278,EFO:0010961
AAACCGGGTAGGTACC-1,1,42,28,347,453,47,10.375277,53,11.699780,AAACCGGGTAGGTACC-1,...,0.000547,0.017277,0.032265,0.819287,4,3.862053,3.408283,0.243512,0.000508,EFO:0010961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,185,210,6,2.857143,18,8.571429,TTGTTCAGTGTGCTAC-1,...,0.000940,0.010030,0.140575,0.026521,1,0.603950,0.187112,0.014475,0.001555,EFO:0010961
TTGTTGTGTGTCAAGA-1,1,31,77,143,175,4,2.285714,18,10.285714,TTGTTGTGTGTCAAGA-1,...,0.002091,0.008887,0.032770,0.001563,3,0.266120,0.018888,0.014429,0.010205,EFO:0010961
TTGTTTCACATCCAGG-1,1,58,42,75,79,6,7.594936,5,6.329114,TTGTTTCACATCCAGG-1,...,0.000330,0.002047,0.003116,0.010596,0,0.520835,0.022958,0.005394,0.000805,EFO:0010961


#### **cell_type_ontology_term_id**

In [80]:
#get the column in adata.obs related. to cell type annotation

In [81]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id'],
      dtype='object')

In [82]:
adata.obsm

AxisArrays with keys: X_spatial

In [83]:
c2l_columns = [col for col in adata.obs.columns if col.startswith('c2l')]

In [84]:
c2l_columns

['c2l_Th',
 'c2l_NK',
 'c2l_APOD+ fibroblasts',
 'c2l_CD8+ T RM',
 'c2l_T reg',
 'c2l_Macro1_2',
 'c2l_DC1',
 'c2l_SFRP2+ fibroblasts',
 'c2l_TAGLN+ pericytes',
 'c2l_POSTN+ fibroblasts',
 'c2l_RGS5+ pericytes',
 'c2l_VEC',
 'c2l_Tc',
 'c2l_ILC_NK',
 'c2l_BC',
 'c2l_Monocytes',
 'c2l_MastC',
 'c2l_Melanocytes',
 'c2l_DC2',
 'c2l_LEC',
 'c2l_PlasmaC',
 'c2l_PTGDS+ fibroblasts',
 'c2l_MigDC',
 'c2l_Neuronal_SchwannC',
 'c2l_SMC',
 'c2l_Skeletal muscle cells',
 'c2l_Suprabasal keratinocytes',
 'c2l_Basal keratinocytes',
 'c2l_Chondrocytes',
 'c2l_IL8+ DC1']

In [85]:
adata.obs['max_c2l_column'] = adata.obs[c2l_columns].idxmax(axis=1)

In [86]:
adata.obs['max_c2l_column_value'] = adata.obs[c2l_columns].max(axis=1)

In [87]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,173,195,8,4.102564,15,7.692308,AAACACCAATAACTGC-1,...,0.024605,0.006612,3,0.171487,1.714548,0.061199,0.005679,EFO:0010961,c2l_Basal keratinocytes,1.714548
AAACAGCTTTCAGAAG-1,1,43,9,216,261,19,7.279693,50,19.157087,AAACAGCTTTCAGAAG-1,...,0.002446,0.347454,1,4.056033,5.330975,0.054002,0.001896,EFO:0010961,c2l_Basal keratinocytes,5.330975
AAACAGGGTCTATATT-1,1,47,13,494,668,38,5.688622,90,13.473053,AAACAGGGTCTATATT-1,...,0.065812,1.291950,4,6.572667,2.795006,0.270538,0.001760,EFO:0010961,c2l_Suprabasal keratinocytes,6.572667
AAACATGGTGAGAGGA-1,1,62,0,435,523,22,4.206501,51,9.751434,AAACATGGTGAGAGGA-1,...,0.081424,0.027660,5,0.172046,1.047911,0.270653,0.004278,EFO:0010961,c2l_SFRP2+ fibroblasts,2.996142
AAACCGGGTAGGTACC-1,1,42,28,347,453,47,10.375277,53,11.699780,AAACCGGGTAGGTACC-1,...,0.032265,0.819287,4,3.862053,3.408283,0.243512,0.000508,EFO:0010961,c2l_Suprabasal keratinocytes,3.862053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,185,210,6,2.857143,18,8.571429,TTGTTCAGTGTGCTAC-1,...,0.140575,0.026521,1,0.603950,0.187112,0.014475,0.001555,EFO:0010961,c2l_Suprabasal keratinocytes,0.603950
TTGTTGTGTGTCAAGA-1,1,31,77,143,175,4,2.285714,18,10.285714,TTGTTGTGTGTCAAGA-1,...,0.032770,0.001563,3,0.266120,0.018888,0.014429,0.010205,EFO:0010961,c2l_SFRP2+ fibroblasts,5.989968
TTGTTTCACATCCAGG-1,1,58,42,75,79,6,7.594936,5,6.329114,TTGTTTCACATCCAGG-1,...,0.003116,0.010596,0,0.520835,0.022958,0.005394,0.000805,EFO:0010961,c2l_SFRP2+ fibroblasts,0.974798


In [88]:
mapping= {'c2l_Th' :'CL:0000912',
 'c2l_NK':'CL:0000623',
 'c2l_APOD+ fibroblasts':'CL:0000057',
 'c2l_CD8+ T RM':'CL:0001203' ,
 'c2l_T reg':'CL:0000815',
 'c2l_Macro1_2':'CL:0000235',
 'c2l_DC1':'CL:0000990',
 'c2l_SFRP2+ fibroblasts':'CL:0000057',
 'c2l_TAGLN+ pericytes':'CL:0000669',
 'c2l_POSTN+ fibroblasts':'CL:0000057',
 'c2l_RGS5+ pericytes':'CL:0000669',
 'c2l_VEC':'CL:0002139',
 'c2l_Tc':'CL:0000910',
 'c2l_ILC_NK':'CL:0001065', #not NK
 'c2l_BC':'CL:0000646',
 'c2l_Monocytes':'CL:0000576',
 'c2l_MastC':'CL:0000097',
 'c2l_Melanocytes':'CL:1000458',
 'c2l_DC2':'CL:0000784',
 'c2l_LEC':'CL:0002138',
 'c2l_PlasmaC':'CL:0000786',
 'c2l_PTGDS+ fibroblasts':'CL:0000057',
 'c2l_MigDC':'CL:0000451', #not mig
 'c2l_Neuronal_SchwannC':'CL:0002573',
 'c2l_SMC':'CL:0000192',
 'c2l_Skeletal muscle cells':'CL:0000188',
 'c2l_Suprabasal keratinocytes':'CL:4033013',
 'c2l_Basal keratinocytes':'CL:0002187',
 'c2l_Chondrocytes':'CL:0000138',
 'c2l_IL8+ DC1':'CL:0000990'}

In [89]:
# create a dictionary of cell type and ontology term

In [90]:
# add the cell_type_ontology_term_id column

In [91]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['max_c2l_column'].map(mapping)

In [92]:
# change datatype of the column

In [93]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['cell_type_ontology_term_id'].astype('category')

In [94]:
list(adata.obs['cell_type_ontology_term_id'].unique())

['CL:0002187',
 'CL:4033013',
 'CL:0000057',
 'CL:0002139',
 'CL:0002573',
 'CL:0000192',
 'CL:0000138',
 'CL:0000815',
 'CL:0000669',
 'CL:0000646',
 'CL:0000784']

In [95]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,173,195,8,4.102564,15,7.692308,AAACACCAATAACTGC-1,...,0.006612,3,0.171487,1.714548,0.061199,0.005679,EFO:0010961,c2l_Basal keratinocytes,1.714548,CL:0002187
AAACAGCTTTCAGAAG-1,1,43,9,216,261,19,7.279693,50,19.157087,AAACAGCTTTCAGAAG-1,...,0.347454,1,4.056033,5.330975,0.054002,0.001896,EFO:0010961,c2l_Basal keratinocytes,5.330975,CL:0002187
AAACAGGGTCTATATT-1,1,47,13,494,668,38,5.688622,90,13.473053,AAACAGGGTCTATATT-1,...,1.291950,4,6.572667,2.795006,0.270538,0.001760,EFO:0010961,c2l_Suprabasal keratinocytes,6.572667,CL:4033013
AAACATGGTGAGAGGA-1,1,62,0,435,523,22,4.206501,51,9.751434,AAACATGGTGAGAGGA-1,...,0.027660,5,0.172046,1.047911,0.270653,0.004278,EFO:0010961,c2l_SFRP2+ fibroblasts,2.996142,CL:0000057
AAACCGGGTAGGTACC-1,1,42,28,347,453,47,10.375277,53,11.699780,AAACCGGGTAGGTACC-1,...,0.819287,4,3.862053,3.408283,0.243512,0.000508,EFO:0010961,c2l_Suprabasal keratinocytes,3.862053,CL:4033013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,185,210,6,2.857143,18,8.571429,TTGTTCAGTGTGCTAC-1,...,0.026521,1,0.603950,0.187112,0.014475,0.001555,EFO:0010961,c2l_Suprabasal keratinocytes,0.603950,CL:4033013
TTGTTGTGTGTCAAGA-1,1,31,77,143,175,4,2.285714,18,10.285714,TTGTTGTGTGTCAAGA-1,...,0.001563,3,0.266120,0.018888,0.014429,0.010205,EFO:0010961,c2l_SFRP2+ fibroblasts,5.989968,CL:0000057
TTGTTTCACATCCAGG-1,1,58,42,75,79,6,7.594936,5,6.329114,TTGTTTCACATCCAGG-1,...,0.010596,0,0.520835,0.022958,0.005394,0.000805,EFO:0010961,c2l_SFRP2+ fibroblasts,0.974798,CL:0000057


In [96]:
araw.obs

""
AAACACCAATAACTGC-1
AAACAGCTTTCAGAAG-1
AAACAGGGTCTATATT-1
AAACATGGTGAGAGGA-1
AAACCGGGTAGGTACC-1
...
TTGTTCAGTGTGCTAC-1
TTGTTGTGTGTCAAGA-1
TTGTTTCACATCCAGG-1
TTGTTTCATTAGTCTA-1


#### **donor_id**

In [97]:
#identify the column in adata.obs which provides donor information

In [98]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id'],
      dtype='object')

In [99]:
# add the donor_id column

In [100]:
adata.obs['donor_id'] = ['body_back_A'] * len(adata.obs)

In [101]:
# change datatype of the column

In [102]:
adata.obs['donor_id'] = adata.obs['donor_id'].astype('category')

In [103]:
# view unique values of donor_id column

In [104]:
list(adata.obs['donor_id'].unique())

['body_back_A']

In [105]:
#view obs

In [106]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,173,195,8,4.102564,15,7.692308,AAACACCAATAACTGC-1,...,3,0.171487,1.714548,0.061199,0.005679,EFO:0010961,c2l_Basal keratinocytes,1.714548,CL:0002187,body_back_A
AAACAGCTTTCAGAAG-1,1,43,9,216,261,19,7.279693,50,19.157087,AAACAGCTTTCAGAAG-1,...,1,4.056033,5.330975,0.054002,0.001896,EFO:0010961,c2l_Basal keratinocytes,5.330975,CL:0002187,body_back_A
AAACAGGGTCTATATT-1,1,47,13,494,668,38,5.688622,90,13.473053,AAACAGGGTCTATATT-1,...,4,6.572667,2.795006,0.270538,0.001760,EFO:0010961,c2l_Suprabasal keratinocytes,6.572667,CL:4033013,body_back_A
AAACATGGTGAGAGGA-1,1,62,0,435,523,22,4.206501,51,9.751434,AAACATGGTGAGAGGA-1,...,5,0.172046,1.047911,0.270653,0.004278,EFO:0010961,c2l_SFRP2+ fibroblasts,2.996142,CL:0000057,body_back_A
AAACCGGGTAGGTACC-1,1,42,28,347,453,47,10.375277,53,11.699780,AAACCGGGTAGGTACC-1,...,4,3.862053,3.408283,0.243512,0.000508,EFO:0010961,c2l_Suprabasal keratinocytes,3.862053,CL:4033013,body_back_A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,185,210,6,2.857143,18,8.571429,TTGTTCAGTGTGCTAC-1,...,1,0.603950,0.187112,0.014475,0.001555,EFO:0010961,c2l_Suprabasal keratinocytes,0.603950,CL:4033013,body_back_A
TTGTTGTGTGTCAAGA-1,1,31,77,143,175,4,2.285714,18,10.285714,TTGTTGTGTGTCAAGA-1,...,3,0.266120,0.018888,0.014429,0.010205,EFO:0010961,c2l_SFRP2+ fibroblasts,5.989968,CL:0000057,body_back_A
TTGTTTCACATCCAGG-1,1,58,42,75,79,6,7.594936,5,6.329114,TTGTTTCACATCCAGG-1,...,0,0.520835,0.022958,0.005394,0.000805,EFO:0010961,c2l_SFRP2+ fibroblasts,0.974798,CL:0000057,body_back_A


In [107]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id', 'donor_id'],
      dtype='object')

#### **development_stage_ontology_term_id**

In [108]:
# identify the column in adata which corresponds to age

In [109]:
# add the development_stage_ontology_term_id column

In [110]:
adata.obs['development_stage_ontology_term_id'] = ['HsapDv:0000154'] * len(adata.obs)

In [111]:
# change datatype of the column

In [112]:
adata.obs['development_stage_ontology_term_id'] = adata.obs['development_stage_ontology_term_id'].astype('category')

In [113]:
# view unique values of development_stage_ontology_term_id column

In [114]:
list(adata.obs['development_stage_ontology_term_id'].unique())

['HsapDv:0000154']

In [115]:
# view adata.obs

In [116]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,173,195,8,4.102564,15,7.692308,AAACACCAATAACTGC-1,...,0.171487,1.714548,0.061199,0.005679,EFO:0010961,c2l_Basal keratinocytes,1.714548,CL:0002187,body_back_A,HsapDv:0000154
AAACAGCTTTCAGAAG-1,1,43,9,216,261,19,7.279693,50,19.157087,AAACAGCTTTCAGAAG-1,...,4.056033,5.330975,0.054002,0.001896,EFO:0010961,c2l_Basal keratinocytes,5.330975,CL:0002187,body_back_A,HsapDv:0000154
AAACAGGGTCTATATT-1,1,47,13,494,668,38,5.688622,90,13.473053,AAACAGGGTCTATATT-1,...,6.572667,2.795006,0.270538,0.001760,EFO:0010961,c2l_Suprabasal keratinocytes,6.572667,CL:4033013,body_back_A,HsapDv:0000154
AAACATGGTGAGAGGA-1,1,62,0,435,523,22,4.206501,51,9.751434,AAACATGGTGAGAGGA-1,...,0.172046,1.047911,0.270653,0.004278,EFO:0010961,c2l_SFRP2+ fibroblasts,2.996142,CL:0000057,body_back_A,HsapDv:0000154
AAACCGGGTAGGTACC-1,1,42,28,347,453,47,10.375277,53,11.699780,AAACCGGGTAGGTACC-1,...,3.862053,3.408283,0.243512,0.000508,EFO:0010961,c2l_Suprabasal keratinocytes,3.862053,CL:4033013,body_back_A,HsapDv:0000154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,185,210,6,2.857143,18,8.571429,TTGTTCAGTGTGCTAC-1,...,0.603950,0.187112,0.014475,0.001555,EFO:0010961,c2l_Suprabasal keratinocytes,0.603950,CL:4033013,body_back_A,HsapDv:0000154
TTGTTGTGTGTCAAGA-1,1,31,77,143,175,4,2.285714,18,10.285714,TTGTTGTGTGTCAAGA-1,...,0.266120,0.018888,0.014429,0.010205,EFO:0010961,c2l_SFRP2+ fibroblasts,5.989968,CL:0000057,body_back_A,HsapDv:0000154
TTGTTTCACATCCAGG-1,1,58,42,75,79,6,7.594936,5,6.329114,TTGTTTCACATCCAGG-1,...,0.520835,0.022958,0.005394,0.000805,EFO:0010961,c2l_SFRP2+ fibroblasts,0.974798,CL:0000057,body_back_A,HsapDv:0000154


#### **disease_ontology_term_id**

In [117]:
# Assign normal since all are healthy patients

In [118]:
# add the disease_ontology_term_id column

In [119]:
adata.obs['disease_ontology_term_id'] = ['PATO:0000461'] * len(adata.obs)

In [120]:
#change data type of column

In [121]:
adata.obs['disease_ontology_term_id'] = adata.obs['disease_ontology_term_id'].astype('category')

In [122]:
# view obs

In [123]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,173,195,8,4.102564,15,7.692308,AAACACCAATAACTGC-1,...,1.714548,0.061199,0.005679,EFO:0010961,c2l_Basal keratinocytes,1.714548,CL:0002187,body_back_A,HsapDv:0000154,PATO:0000461
AAACAGCTTTCAGAAG-1,1,43,9,216,261,19,7.279693,50,19.157087,AAACAGCTTTCAGAAG-1,...,5.330975,0.054002,0.001896,EFO:0010961,c2l_Basal keratinocytes,5.330975,CL:0002187,body_back_A,HsapDv:0000154,PATO:0000461
AAACAGGGTCTATATT-1,1,47,13,494,668,38,5.688622,90,13.473053,AAACAGGGTCTATATT-1,...,2.795006,0.270538,0.001760,EFO:0010961,c2l_Suprabasal keratinocytes,6.572667,CL:4033013,body_back_A,HsapDv:0000154,PATO:0000461
AAACATGGTGAGAGGA-1,1,62,0,435,523,22,4.206501,51,9.751434,AAACATGGTGAGAGGA-1,...,1.047911,0.270653,0.004278,EFO:0010961,c2l_SFRP2+ fibroblasts,2.996142,CL:0000057,body_back_A,HsapDv:0000154,PATO:0000461
AAACCGGGTAGGTACC-1,1,42,28,347,453,47,10.375277,53,11.699780,AAACCGGGTAGGTACC-1,...,3.408283,0.243512,0.000508,EFO:0010961,c2l_Suprabasal keratinocytes,3.862053,CL:4033013,body_back_A,HsapDv:0000154,PATO:0000461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,185,210,6,2.857143,18,8.571429,TTGTTCAGTGTGCTAC-1,...,0.187112,0.014475,0.001555,EFO:0010961,c2l_Suprabasal keratinocytes,0.603950,CL:4033013,body_back_A,HsapDv:0000154,PATO:0000461
TTGTTGTGTGTCAAGA-1,1,31,77,143,175,4,2.285714,18,10.285714,TTGTTGTGTGTCAAGA-1,...,0.018888,0.014429,0.010205,EFO:0010961,c2l_SFRP2+ fibroblasts,5.989968,CL:0000057,body_back_A,HsapDv:0000154,PATO:0000461
TTGTTTCACATCCAGG-1,1,58,42,75,79,6,7.594936,5,6.329114,TTGTTTCACATCCAGG-1,...,0.022958,0.005394,0.000805,EFO:0010961,c2l_SFRP2+ fibroblasts,0.974798,CL:0000057,body_back_A,HsapDv:0000154,PATO:0000461


#### **is_primary_data**

In [124]:
adata.obs['is_primary_data'] = [True] * len(adata.obs)

In [125]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,173,195,8,4.102564,15,7.692308,AAACACCAATAACTGC-1,...,0.061199,0.005679,EFO:0010961,c2l_Basal keratinocytes,1.714548,CL:0002187,body_back_A,HsapDv:0000154,PATO:0000461,True
AAACAGCTTTCAGAAG-1,1,43,9,216,261,19,7.279693,50,19.157087,AAACAGCTTTCAGAAG-1,...,0.054002,0.001896,EFO:0010961,c2l_Basal keratinocytes,5.330975,CL:0002187,body_back_A,HsapDv:0000154,PATO:0000461,True
AAACAGGGTCTATATT-1,1,47,13,494,668,38,5.688622,90,13.473053,AAACAGGGTCTATATT-1,...,0.270538,0.001760,EFO:0010961,c2l_Suprabasal keratinocytes,6.572667,CL:4033013,body_back_A,HsapDv:0000154,PATO:0000461,True
AAACATGGTGAGAGGA-1,1,62,0,435,523,22,4.206501,51,9.751434,AAACATGGTGAGAGGA-1,...,0.270653,0.004278,EFO:0010961,c2l_SFRP2+ fibroblasts,2.996142,CL:0000057,body_back_A,HsapDv:0000154,PATO:0000461,True
AAACCGGGTAGGTACC-1,1,42,28,347,453,47,10.375277,53,11.699780,AAACCGGGTAGGTACC-1,...,0.243512,0.000508,EFO:0010961,c2l_Suprabasal keratinocytes,3.862053,CL:4033013,body_back_A,HsapDv:0000154,PATO:0000461,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,185,210,6,2.857143,18,8.571429,TTGTTCAGTGTGCTAC-1,...,0.014475,0.001555,EFO:0010961,c2l_Suprabasal keratinocytes,0.603950,CL:4033013,body_back_A,HsapDv:0000154,PATO:0000461,True
TTGTTGTGTGTCAAGA-1,1,31,77,143,175,4,2.285714,18,10.285714,TTGTTGTGTGTCAAGA-1,...,0.014429,0.010205,EFO:0010961,c2l_SFRP2+ fibroblasts,5.989968,CL:0000057,body_back_A,HsapDv:0000154,PATO:0000461,True
TTGTTTCACATCCAGG-1,1,58,42,75,79,6,7.594936,5,6.329114,TTGTTTCACATCCAGG-1,...,0.005394,0.000805,EFO:0010961,c2l_SFRP2+ fibroblasts,0.974798,CL:0000057,body_back_A,HsapDv:0000154,PATO:0000461,True


In [126]:
#change data type of column

In [127]:
adata.obs['is_primary_data'] = adata.obs['is_primary_data'].astype('bool')

#### **organism_ontology_term_id**

In [128]:
# assign organism id 

In [129]:
adata.obs['organism_ontology_term_id'] = ['NCBITaxon:9606'] * len(adata.obs)

In [130]:
#change data type of column

In [131]:
adata.obs['organism_ontology_term_id'] = adata.obs['organism_ontology_term_id'].astype('category')

In [132]:
# view obs

In [133]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,173,195,8,4.102564,15,7.692308,AAACACCAATAACTGC-1,...,0.005679,EFO:0010961,c2l_Basal keratinocytes,1.714548,CL:0002187,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606
AAACAGCTTTCAGAAG-1,1,43,9,216,261,19,7.279693,50,19.157087,AAACAGCTTTCAGAAG-1,...,0.001896,EFO:0010961,c2l_Basal keratinocytes,5.330975,CL:0002187,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606
AAACAGGGTCTATATT-1,1,47,13,494,668,38,5.688622,90,13.473053,AAACAGGGTCTATATT-1,...,0.001760,EFO:0010961,c2l_Suprabasal keratinocytes,6.572667,CL:4033013,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606
AAACATGGTGAGAGGA-1,1,62,0,435,523,22,4.206501,51,9.751434,AAACATGGTGAGAGGA-1,...,0.004278,EFO:0010961,c2l_SFRP2+ fibroblasts,2.996142,CL:0000057,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606
AAACCGGGTAGGTACC-1,1,42,28,347,453,47,10.375277,53,11.699780,AAACCGGGTAGGTACC-1,...,0.000508,EFO:0010961,c2l_Suprabasal keratinocytes,3.862053,CL:4033013,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,185,210,6,2.857143,18,8.571429,TTGTTCAGTGTGCTAC-1,...,0.001555,EFO:0010961,c2l_Suprabasal keratinocytes,0.603950,CL:4033013,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606
TTGTTGTGTGTCAAGA-1,1,31,77,143,175,4,2.285714,18,10.285714,TTGTTGTGTGTCAAGA-1,...,0.010205,EFO:0010961,c2l_SFRP2+ fibroblasts,5.989968,CL:0000057,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606
TTGTTTCACATCCAGG-1,1,58,42,75,79,6,7.594936,5,6.329114,TTGTTTCACATCCAGG-1,...,0.000805,EFO:0010961,c2l_SFRP2+ fibroblasts,0.974798,CL:0000057,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606


#### **self_reported_ethnicity_ontology_term_id**

In [134]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = ['unknown'] * len(adata.obs)

In [135]:
# change data type

In [136]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = adata.obs['self_reported_ethnicity_ontology_term_id'].astype('category')

In [137]:
# view obs

In [138]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,173,195,8,4.102564,15,7.692308,AAACACCAATAACTGC-1,...,EFO:0010961,c2l_Basal keratinocytes,1.714548,CL:0002187,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown
AAACAGCTTTCAGAAG-1,1,43,9,216,261,19,7.279693,50,19.157087,AAACAGCTTTCAGAAG-1,...,EFO:0010961,c2l_Basal keratinocytes,5.330975,CL:0002187,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown
AAACAGGGTCTATATT-1,1,47,13,494,668,38,5.688622,90,13.473053,AAACAGGGTCTATATT-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,6.572667,CL:4033013,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown
AAACATGGTGAGAGGA-1,1,62,0,435,523,22,4.206501,51,9.751434,AAACATGGTGAGAGGA-1,...,EFO:0010961,c2l_SFRP2+ fibroblasts,2.996142,CL:0000057,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown
AAACCGGGTAGGTACC-1,1,42,28,347,453,47,10.375277,53,11.699780,AAACCGGGTAGGTACC-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,3.862053,CL:4033013,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,185,210,6,2.857143,18,8.571429,TTGTTCAGTGTGCTAC-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,0.603950,CL:4033013,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown
TTGTTGTGTGTCAAGA-1,1,31,77,143,175,4,2.285714,18,10.285714,TTGTTGTGTGTCAAGA-1,...,EFO:0010961,c2l_SFRP2+ fibroblasts,5.989968,CL:0000057,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown
TTGTTTCACATCCAGG-1,1,58,42,75,79,6,7.594936,5,6.329114,TTGTTTCACATCCAGG-1,...,EFO:0010961,c2l_SFRP2+ fibroblasts,0.974798,CL:0000057,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown


In [139]:
list(adata.obs['self_reported_ethnicity_ontology_term_id'].unique())

['unknown']

#### **sex_ontology_term_id**

In [140]:
adata.obs['sex_ontology_term_id'] = ['PATO:0000384'] * len(adata.obs)

In [141]:
# change data type

In [142]:
adata.obs['sex_ontology_term_id'] = adata.obs['sex_ontology_term_id'].astype('category')

In [143]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,173,195,8,4.102564,15,7.692308,AAACACCAATAACTGC-1,...,c2l_Basal keratinocytes,1.714548,CL:0002187,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
AAACAGCTTTCAGAAG-1,1,43,9,216,261,19,7.279693,50,19.157087,AAACAGCTTTCAGAAG-1,...,c2l_Basal keratinocytes,5.330975,CL:0002187,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
AAACAGGGTCTATATT-1,1,47,13,494,668,38,5.688622,90,13.473053,AAACAGGGTCTATATT-1,...,c2l_Suprabasal keratinocytes,6.572667,CL:4033013,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
AAACATGGTGAGAGGA-1,1,62,0,435,523,22,4.206501,51,9.751434,AAACATGGTGAGAGGA-1,...,c2l_SFRP2+ fibroblasts,2.996142,CL:0000057,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
AAACCGGGTAGGTACC-1,1,42,28,347,453,47,10.375277,53,11.699780,AAACCGGGTAGGTACC-1,...,c2l_Suprabasal keratinocytes,3.862053,CL:4033013,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,185,210,6,2.857143,18,8.571429,TTGTTCAGTGTGCTAC-1,...,c2l_Suprabasal keratinocytes,0.603950,CL:4033013,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
TTGTTGTGTGTCAAGA-1,1,31,77,143,175,4,2.285714,18,10.285714,TTGTTGTGTGTCAAGA-1,...,c2l_SFRP2+ fibroblasts,5.989968,CL:0000057,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
TTGTTTCACATCCAGG-1,1,58,42,75,79,6,7.594936,5,6.329114,TTGTTTCACATCCAGG-1,...,c2l_SFRP2+ fibroblasts,0.974798,CL:0000057,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384


#### **suspension_type**

In [144]:
# since visium suspension type is 'na'

In [145]:
adata.obs['suspension_type'] = ['na'] * len(adata.obs)

In [146]:
# change data type

In [147]:
adata.obs['suspension_type'] = adata.obs['suspension_type'].astype('category')

In [148]:
# view obs

In [149]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,173,195,8,4.102564,15,7.692308,AAACACCAATAACTGC-1,...,1.714548,CL:0002187,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACAGCTTTCAGAAG-1,1,43,9,216,261,19,7.279693,50,19.157087,AAACAGCTTTCAGAAG-1,...,5.330975,CL:0002187,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACAGGGTCTATATT-1,1,47,13,494,668,38,5.688622,90,13.473053,AAACAGGGTCTATATT-1,...,6.572667,CL:4033013,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACATGGTGAGAGGA-1,1,62,0,435,523,22,4.206501,51,9.751434,AAACATGGTGAGAGGA-1,...,2.996142,CL:0000057,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACCGGGTAGGTACC-1,1,42,28,347,453,47,10.375277,53,11.699780,AAACCGGGTAGGTACC-1,...,3.862053,CL:4033013,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,185,210,6,2.857143,18,8.571429,TTGTTCAGTGTGCTAC-1,...,0.603950,CL:4033013,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
TTGTTGTGTGTCAAGA-1,1,31,77,143,175,4,2.285714,18,10.285714,TTGTTGTGTGTCAAGA-1,...,5.989968,CL:0000057,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
TTGTTTCACATCCAGG-1,1,58,42,75,79,6,7.594936,5,6.329114,TTGTTTCACATCCAGG-1,...,0.974798,CL:0000057,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na


#### **tissue_type**

In [150]:
adata.obs['tissue_type'] = ['tissue'] * len(adata.obs)

In [151]:
adata.obs['tissue_type'] = adata.obs['tissue_type'].astype('category')

#### **tissue_ontology_term_id**

In [152]:
adata.obs['tissue_ontology_term_id'] = adata.obs['donor_id'].map(mapping)

In [153]:
adata.obs['tissue_ontology_term_id'] = ['UBERON:0001068'] * len(adata.obs)

In [154]:
list(adata.obs['tissue_ontology_term_id'].unique())

['UBERON:0001068']

In [155]:
# view obs

In [156]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,59,19,173,195,8,4.102564,15,7.692308,AAACACCAATAACTGC-1,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
AAACAGCTTTCAGAAG-1,1,43,9,216,261,19,7.279693,50,19.157087,AAACAGCTTTCAGAAG-1,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
AAACAGGGTCTATATT-1,1,47,13,494,668,38,5.688622,90,13.473053,AAACAGGGTCTATATT-1,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
AAACATGGTGAGAGGA-1,1,62,0,435,523,22,4.206501,51,9.751434,AAACATGGTGAGAGGA-1,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
AAACCGGGTAGGTACC-1,1,42,28,347,453,47,10.375277,53,11.699780,AAACCGGGTAGGTACC-1,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,24,64,185,210,6,2.857143,18,8.571429,TTGTTCAGTGTGCTAC-1,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
TTGTTGTGTGTCAAGA-1,1,31,77,143,175,4,2.285714,18,10.285714,TTGTTGTGTGTCAAGA-1,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
TTGTTTCACATCCAGG-1,1,58,42,75,79,6,7.594936,5,6.329114,TTGTTTCACATCCAGG-1,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068


In [157]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 

In [158]:
del adata.obs['barcode']
del adata.obs['max_c2l_column']
del adata.obs['max_c2l_column_value']
del adata.obs['array_row']
del adata.obs['array_col']

#### **obsm (Embeddings)**

In [159]:
adata.obsm

AxisArrays with keys: X_spatial

In [160]:
adata.obsm.keys()

KeysView(AxisArrays with keys: X_spatial)

#### **uns (Dataset Metadata)**

In [161]:
adata.uns

OverloadedDict, wrapping:
	{'leiden': {'params': {'n_iterations': -1, 'random_state': 0, 'resolution': 1}}, 'log1p': {}, 'neighbors': {'connectivities_key': 'connectivities', 'distances_key': 'distances', 'params': {'method': 'umap', 'metric': 'euclidean', 'n_neighbors': 15, 'random_state': 0}}, 'pca': {'params': {'use_highly_variable': False, 'zero_center': True}, 'variance': array([412.05832  ,  56.356342 ,  44.346684 ,  30.470612 ,  26.300024 ,
        23.240955 ,  20.516842 ,  19.975204 ,  18.902567 ,  17.999783 ,
        17.633753 ,  17.265106 ,  17.175522 ,  17.106012 ,  16.92194  ,
        16.843212 ,  16.769785 ,  16.67691  ,  16.61374  ,  16.565832 ,
        16.47174  ,  16.33099  ,  16.315748 ,  16.268274 ,  16.242908 ,
        16.213198 ,  16.135925 ,  16.097221 ,  16.051556 ,  16.027273 ,
        16.000593 ,  15.953295 ,  15.91392  ,  15.863666 ,  15.815144 ,
        15.76766  ,  15.755058 ,  15.730999 ,  15.662213 ,  15.627175 ,
        15.59794  ,  15.586675 ,  15.538353 

In [162]:
adata.uns['image_caption'] = 'Shown here is an image of 10 μm thick slice of the skin from the back stained with H&E'

In [163]:
adata.uns['title'] = 'Visium spatial - body_back_A_a'

In [164]:
adata.uns['default_embedding'] = 'X_spatial'

In [165]:
adata.uns.keys()

dict_keys(['leiden', 'log1p', 'neighbors', 'pca', 'spatial', 'umap', 'image_caption', 'title', 'default_embedding'])

### **Final checks and adjustments**

In [166]:
adata

AnnData object with n_obs × n_vars = 2418 × 36390
    obs: 'in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes', 'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc', 'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes', 'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts', 'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC', 'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes', 'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'tissue_t

In [167]:
adata.obs.dtypes

in_tissue                                      int64
n_genes_by_counts                              int32
total_counts                                   int32
total_counts_mt                                int32
pct_counts_mt                                float32
total_counts_rb                                int32
pct_counts_rb                                float32
c2l_Th                                       float64
c2l_NK                                       float64
c2l_APOD+ fibroblasts                        float64
c2l_CD8+ T RM                                float64
c2l_T reg                                    float64
c2l_Macro1_2                                 float64
c2l_DC1                                      float64
c2l_SFRP2+ fibroblasts                       float64
c2l_TAGLN+ pericytes                         float64
c2l_POSTN+ fibroblasts                       float64
c2l_RGS5+ pericytes                          float64
c2l_VEC                                      f

In [168]:
dty = pd.DataFrame(adata.var.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.var[c] = adata.var[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.var[c] = adata.var[c].astype('int32') 
    print(f"changed {c} from int64 to int32")

In [169]:
dty = pd.DataFrame(adata.obs.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.obs[c] = adata.obs[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.obs[c] = adata.obs[c].astype('int32') 
    print(f"changed {c} from int64 to int32")
for c in dty[dty['dtype'] == 'object'].index.values:
    adata.obs[c] = adata.obs[c].astype('category') 
    print(f"changed {c} from object to category")

changed c2l_Th from float64 to float32
changed c2l_NK from float64 to float32
changed c2l_APOD+ fibroblasts from float64 to float32
changed c2l_CD8+ T RM from float64 to float32
changed c2l_T reg from float64 to float32
changed c2l_Macro1_2 from float64 to float32
changed c2l_DC1 from float64 to float32
changed c2l_SFRP2+ fibroblasts from float64 to float32
changed c2l_TAGLN+ pericytes from float64 to float32
changed c2l_POSTN+ fibroblasts from float64 to float32
changed c2l_RGS5+ pericytes from float64 to float32
changed c2l_VEC from float64 to float32
changed c2l_Tc from float64 to float32
changed c2l_ILC_NK from float64 to float32
changed c2l_BC from float64 to float32
changed c2l_Monocytes from float64 to float32
changed c2l_MastC from float64 to float32
changed c2l_Melanocytes from float64 to float32
changed c2l_DC2 from float64 to float32
changed c2l_LEC from float64 to float32
changed c2l_PlasmaC from float64 to float32
changed c2l_PTGDS+ fibroblasts from float64 to float32
chan

In [170]:
adata.obs

,in_tissue,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,c2l_Th,c2l_NK,c2l_APOD+ fibroblasts,...,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,173,195,8,4.102564,15,7.692308,0.003839,0.051966,0.161906,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
AAACAGCTTTCAGAAG-1,1,216,261,19,7.279693,50,19.157087,0.003553,0.022013,0.023782,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
AAACAGGGTCTATATT-1,1,494,668,38,5.688622,90,13.473053,0.008981,0.004897,0.033217,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
AAACATGGTGAGAGGA-1,1,435,523,22,4.206501,51,9.751434,0.007612,0.012887,0.150511,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
AAACCGGGTAGGTACC-1,1,347,453,47,10.375277,53,11.699780,0.003709,0.002913,0.010997,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,185,210,6,2.857143,18,8.571429,0.001325,0.010108,0.111337,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
TTGTTGTGTGTCAAGA-1,1,143,175,4,2.285714,18,10.285714,0.005480,0.022335,0.021961,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
TTGTTTCACATCCAGG-1,1,75,79,6,7.594936,5,6.329114,0.000649,0.010792,0.023179,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068


In [171]:
adata.obs.columns

Index(['in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK',
       'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2',
       'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id', 'sex_ontology

In [172]:
adata.var

,feature_is_filtered
ENSG00000243485,True
ENSG00000237613,True
ENSG00000186092,True
ENSG00000238009,False
ENSG00000239945,True
...,...
ENSG00000277836,True
ENSG00000278633,True
ENSG00000276017,True
ENSG00000278817,False


In [173]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


In [174]:
adata.obs

,in_tissue,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,c2l_Th,c2l_NK,c2l_APOD+ fibroblasts,...,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACACCAATAACTGC-1,1,173,195,8,4.102564,15,7.692308,0.003839,0.051966,0.161906,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
AAACAGCTTTCAGAAG-1,1,216,261,19,7.279693,50,19.157087,0.003553,0.022013,0.023782,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
AAACAGGGTCTATATT-1,1,494,668,38,5.688622,90,13.473053,0.008981,0.004897,0.033217,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
AAACATGGTGAGAGGA-1,1,435,523,22,4.206501,51,9.751434,0.007612,0.012887,0.150511,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
AAACCGGGTAGGTACC-1,1,347,453,47,10.375277,53,11.699780,0.003709,0.002913,0.010997,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTCAGTGTGCTAC-1,1,185,210,6,2.857143,18,8.571429,0.001325,0.010108,0.111337,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
TTGTTGTGTGTCAAGA-1,1,143,175,4,2.285714,18,10.285714,0.005480,0.022335,0.021961,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068
TTGTTTCACATCCAGG-1,1,75,79,6,7.594936,5,6.329114,0.000649,0.010792,0.023179,...,body_back_A,HsapDv:0000154,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0001068


In [175]:
adata.obs.columns

Index(['in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK',
       'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2',
       'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id', 'sex_ontology

In [176]:
#check the format of expression matrix

In [177]:
adata.X

<2418x36390 sparse matrix of type '<class 'numpy.float32'>'
	with 38262432 stored elements in Compressed Sparse Row format>

In [178]:
araw.X

<2418x36390 sparse matrix of type '<class 'numpy.float32'>'
	with 1525819 stored elements in Compressed Sparse Row format>

In [179]:
#Copy raw counts to adata.raw

In [180]:
adata.raw = araw

In [181]:
del adata.uns['log1p']

In [182]:
#write the curated object to final_objects folder

In [183]:
adata.write('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Final_objects/body_back1a.h5ad', compression = 'gzip')